In [1]:
import pandas as pd
import jieba
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import re
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.contrib.layers import fully_connected
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

/home/mike/anaconda3/envs/nlp/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mike/anaconda3/envs/nlp/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mike/anaconda3/envs/nlp/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mike/anaconda3/envs/nlp/lib/python3.6/site-packages/ten

In [2]:
movies = pd.read_csv('./movie_comments.csv')

/home/mike/anaconda3/envs/nlp/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
movies.head()

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [4]:
movies.describe()

,id,link,name,comment,star
count,261497,261497,261497,261495,261497
unique,260150,2761,2760,213970,11
top,11,https://movie.douban.com/subject/1849031/,当幸福来敲门 The Pursuit of Happyness,经典,4
freq,6,396,396,200,43002


In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261497 entries, 0 to 261496
Data columns (total 5 columns):
id         261497 non-null object
link       261497 non-null object
name       261497 non-null object
comment    261495 non-null object
star       261497 non-null object
dtypes: object(5)
memory usage: 10.0+ MB


In [6]:
movies.dropna(axis=0, inplace=True)
movies = movies[movies['star'] != 'star']

In [7]:
movies['star'] = movies['star'].astype('int')

In [8]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261494 entries, 0 to 261496
Data columns (total 5 columns):
id         261494 non-null object
link       261494 non-null object
name       261494 non-null object
comment    261494 non-null object
star       261494 non-null int64
dtypes: int64(1), object(4)
memory usage: 12.0+ MB


In [9]:
movies['star'].value_counts()

4    83808
3    65672
5    59315
2    28136
1    24563
Name: star, dtype: int64

### Cleanning data

In [16]:
def clean_data(string):
    return ''.join(re.findall('\w+', string))

In [17]:
with open('./clean_data.txt', 'w') as f:
    for line in movies['comment']:
        cleaned_data = clean_data(line)
        f.write(cleaned_data + '\n')

### Jieba segmentation

In [18]:
with open('./chinese_stopwords.txt', 'r') as f:
    stop_words = f.read().splitlines()

def cut(string):
    return ' '.join(jieba.cut(string))

def filterStopWords(string):
    strArr = string.split(' ')
    res = ''
    for word in strArr:
        if word not in stop_words:
            res += word + ' '
    if res == '':
        return ''
    else:
        return res[:-1]

In [19]:
with open('./clean_data.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
with open('./cut_data.txt', 'w') as f:
    for line in lines:
        cut_line = cut(line)
        filter_line = filterStopWords(cut_line)
        if filter_line != '':
            f.write(filter_line)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 5.506 seconds.
Prefix dict has been built succesfully.


### training word2vec model

In [20]:
word_embeding_size = 250
sentences = LineSentence('./cut_data.txt')
model = Word2Vec(sentences, size=word_embeding_size, min_count=5, window=5, workers=10)
model.save('./comments_classification')
model.wv.save_word2vec_format('./newsWithCount5.txt', binary=False)

In [21]:
print('语料数：', model.corpus_count)
print('词表长度：', len(model.wv.vocab))

语料数： 259512
词表长度： 40052


### Preparing data for training neural network model

In [10]:
model = Word2Vec.load('./comments_classification')
with open('./cut_data.txt', 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
output = movies['star']
input = []
for index, line in enumerate(lines):
    strArr = line.strip().split(" ")
    res = np.zeros(250)
    len_str_arr = len(strArr)
    for word in strArr:
        if word in model:
            word_t = np.array(model[word])
            res = res + word_t
    if res.all() != 0:
        input.append(res / len_str_arr)
    else:
        output.drop(index=index, inplace=True)

/home/mike/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # This is added back by InteractiveShellApp.init_path()
/home/mike/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [11]:
def reformat(labels, num_labels):
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == (labels[:,None] - 1)).astype(np.int32)
  return labels

In [12]:
input = np.array(input)
output = reformat(output, 5)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(input, output, test_size = 0.2, random_state=42, shuffle=True)

### Training neural network model

In [18]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))) / predictions.shape[0]

In [15]:
n_inputs = 250
n_hidden1 = 200
n_hidden2 = 100
n_outputs = 5
learning_rate = 0.01

graph = tf.Graph()
with graph.as_default():
    
    X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
    y = tf.placeholder(tf.int32, shape=(None, n_outputs), name='y')

    with tf.name_scope('dnn'):
        hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name='hidden1')
        hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name='hidden2')
        logits = tf.layers.dense(hidden2, n_outputs, name='outputs')

    with tf.name_scope("loss"):
        error = tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = logits)
        loss = tf.reduce_mean(error, name = 'loss')
        
    with tf.name_scope("train"):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
        
    
        
    with tf.name_scope("eval"):
        prediction = tf.nn.softmax(logits)
#         correct = tf.nn.in_top_k(logits, y, 1)
#         print(correct)
#         accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [21]:
n_epochs = 20
batch_size = 128
# iteration = 0
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print("Initialized")
    for epoch in range(n_epochs):
        for iteration in range(len(X_train)//batch_size):
            offset = iteration * batch_size
            batch_data = X_train[offset:(offset + batch_size), :]
            batch_labels = y_train[offset:(offset + batch_size), :]
            sess.run([optimizer], feed_dict={X:batch_data, y:batch_labels})
        acc_train = accuracy(prediction.eval(feed_dict={X:batch_data}), batch_labels)
        acc_test = accuracy(prediction.eval(feed_dict={X:X_test}), y_test)
        print("epoch: {}, Train accuracy: {}%, Test accuracy: {}%".format(epoch, acc_train, acc_test))
            

Initialized
epoch: 0, Train accuracy: 35.15625%, Test accuracy: 39.10918516759666%
epoch: 1, Train accuracy: 35.9375%, Test accuracy: 39.79381851280225%
epoch: 2, Train accuracy: 39.0625%, Test accuracy: 40.17768807629902%
epoch: 3, Train accuracy: 38.28125%, Test accuracy: 40.28849578534964%
epoch: 4, Train accuracy: 38.28125%, Test accuracy: 40.36764414895722%
epoch: 5, Train accuracy: 39.0625%, Test accuracy: 40.44481380347462%
epoch: 6, Train accuracy: 39.0625%, Test accuracy: 40.603110530689776%
epoch: 7, Train accuracy: 38.28125%, Test accuracy: 40.67236534884641%
epoch: 8, Train accuracy: 38.28125%, Test accuracy: 40.66247180339546%
epoch: 9, Train accuracy: 38.28125%, Test accuracy: 40.767343385175515%
epoch: 10, Train accuracy: 39.0625%, Test accuracy: 40.846491748783095%
epoch: 11, Train accuracy: 38.28125%, Test accuracy: 40.92959753057106%
epoch: 12, Train accuracy: 38.28125%, Test accuracy: 40.94740591238276%
epoch: 13, Train accuracy: 38.28125%, Test accuracy: 41.00874589